### DATA PREPROCESSING

In [ ]:
# Requirments
!pip install openpyxl
!pip install tqdm

In [33]:
# Modules
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

## Data Exploration

In [34]:
# Looking brief insight
data= pd.read_excel('DataX.xlsx')
print('There are {} rows and {} columns in our dataset.'.format(data.shape[0],data.shape[1]))

There are 119738 rows and 15 columns in our dataset.


In [36]:
print(data.dtypes)

fit                object
user_id             int64
bust size          object
item_id             int64
weight             object
rating            float64
rented for         object
review_text        object
body type          object
review_summary     object
category           object
height             object
size                int64
age               float64
review_date        object
dtype: object


In [37]:
data.describe(include=np.number)

,user_id,item_id,rating,size,age
count,119738.000000,1.197380e+05,119688.000000,119738.000000,119134.000000
mean,498679.569694,1.047467e+06,9.094964,12.230720,33.866604
std,289190.859807,8.059424e+05,1.430831,8.497202,8.063488
min,9.000000,1.233730e+05,2.000000,0.000000,0.000000
25%,250016.000000,1.950760e+05,8.000000,8.000000,29.000000
50%,498398.000000,9.569970e+05,10.000000,12.000000,32.000000
75%,750495.500000,1.679360e+06,10.000000,16.000000,37.000000
max,999994.000000,2.966087e+06,10.000000,58.000000,117.000000


In [ ]:
data.describe(include=['object'])

In [39]:
data.isnull().sum()

,0
fit,0
user_id,0
bust size,11418
item_id,0
weight,18603
rating,50
rented for,8
review_text,42
body type,8987
review_summary,232


In [40]:
# Deletion:
data= data.dropna(subset=['age', 'height', 'review_text','review_summary', 'rating', 'rented for'])
data= data.drop(columns=['weight'])
data=data.drop(columns=['body type'])
data=data.drop(columns=['bust size'])
data=data.drop(columns=['height'])


In [41]:
data.head()

,fit,user_id,item_id,rating,rented for,review_text,review_summary,category,size,age,review_date
0,fit,420272,2260466,10.0,vacation,An adorable romper! Belt and zipper were a lit...,So many compliments!,romper,14,28.0,"April 20, 2016"
1,fit,273551,153475,10.0,other,I rented this dress for a photo shoot. The the...,I felt so glamourous!!!,gown,12,36.0,"June 18, 2013"
2,fit,360448,1063761,10.0,party,This hugged in all the right places! It was a ...,It was a great time to celebrate the (almost) ...,sheath,4,116.0,"December 14, 2015"
3,fit,909926,126335,8.0,formal affair,I rented this for my company's black tie award...,Dress arrived on time and in perfect condition.,dress,8,34.0,"February 12, 2014"
4,fit,151944,616682,10.0,wedding,I have always been petite in my upper body and...,Was in love with this dress !!!,gown,12,27.0,"September 26, 2016"


### Feature Transformation and data labeling

In [42]:
#Lableling
fit_mapping = {'fit': 1, 'small': 2, 'arge': 3}
data['fit_encoded'] = data['fit'].map(fit_mapping)
data.head()

,fit,user_id,item_id,rating,rented for,review_text,review_summary,category,size,age,review_date,fit_encoded
0,fit,420272,2260466,10.0,vacation,An adorable romper! Belt and zipper were a lit...,So many compliments!,romper,14,28.0,"April 20, 2016",1.0
1,fit,273551,153475,10.0,other,I rented this dress for a photo shoot. The the...,I felt so glamourous!!!,gown,12,36.0,"June 18, 2013",1.0
2,fit,360448,1063761,10.0,party,This hugged in all the right places! It was a ...,It was a great time to celebrate the (almost) ...,sheath,4,116.0,"December 14, 2015",1.0
3,fit,909926,126335,8.0,formal affair,I rented this for my company's black tie award...,Dress arrived on time and in perfect condition.,dress,8,34.0,"February 12, 2014",1.0
4,fit,151944,616682,10.0,wedding,I have always been petite in my upper body and...,Was in love with this dress !!!,gown,12,27.0,"September 26, 2016",1.0


In [43]:
data['review_date'] = pd.to_datetime(data['review_date'])

In [45]:
data.head()

,fit,user_id,item_id,rating,rented for,review_text,review_summary,category,size,age,review_date,fit_encoded
0,fit,420272,2260466,10.0,vacation,An adorable romper! Belt and zipper were a lit...,So many compliments!,romper,14,28.0,2016-04-20,1.0
1,fit,273551,153475,10.0,other,I rented this dress for a photo shoot. The the...,I felt so glamourous!!!,gown,12,36.0,2013-06-18,1.0
2,fit,360448,1063761,10.0,party,This hugged in all the right places! It was a ...,It was a great time to celebrate the (almost) ...,sheath,4,116.0,2015-12-14,1.0
3,fit,909926,126335,8.0,formal affair,I rented this for my company's black tie award...,Dress arrived on time and in perfect condition.,dress,8,34.0,2014-02-12,1.0
4,fit,151944,616682,10.0,wedding,I have always been petite in my upper body and...,Was in love with this dress !!!,gown,12,27.0,2016-09-26,1.0
